In [1]:
from glob import glob
from medutils.medutils import load_itk, save_itk
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

/home/jjia/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/jjia/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
ori_dir = "/data1/jjia/dataset/lung_function/ori_resolution/SSc_patient_???????_skeleton.mha"
ori_ls = sorted(glob(ori_dir))
len(ori_ls)

333

In [4]:
for img in tqdm(ori_ls):
    img_np, origin, spacing  = load_itk(img, require_ori_sp=True)  # binary vessel image
    out = (img_np>0) * 1 
    save_itk(img.replace('.mha', '_binary.mha'), out, origin, spacing)
    

100%|██████████| 333/333 [29:41<00:00,  5.35s/it]
